# tme6 TAL: Classification supervisée de documents

In [72]:
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import string
import unicodedata
import re
import codecs
from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import svm
import sklearn.naive_bayes
import sklearn.linear_model
from sklearn.metrics import f1_score

## pre-processing


In [2]:
def readAFile(nf):
    '''reads a file and extracts a list of strings 
        representing the ligns of the document.'''
    f = open(nf, 'rb')
    l = []
    txt = f.readlines()
    for i in txt:
        l.append(i.decode("utf-8"))
    f.close()
    return l


def process(txt):
    '''preprocessing of a string:
            * decoding and ascii encoding
            * punctuation elimination
            * letters to lowercase
    '''
    #txt = txt[txt.find("\n\n"):] # elimination de l'entete (on ne conserve que les caractères après la première occurence du motif
    txt = unicodedata.normalize("NFKD",txt).encode("ascii","ignore") # elimination des caractères spéciaux, accents...
    punc = string.punctuation    # recupération de la ponctuation
    punc += u'\n\r\t\\'          # ajouts de caractères à enlever
    table =string.maketrans(punc, ' '*len(punc))  # table de conversion punc -> espace
    txt = string.translate(txt,table).lower() # elimination des accents + minuscules
    #return re.sub(" +"," ", txt) # expression régulière pour transformer les espaces multiples en simples espaces
    return txt

In [27]:
t = readAFile('train.utf8')

#extracting text and labels
x_raw = []
y = []
for txt in t:
    lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt)
    txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
    x_raw.append(txt)
    if('C' in lab):
        bin_lab = 1
    elif('M' in lab):
        bin_lab = 0
    else:
        bin_lab = 'err'
    y.append(bin_lab)

#preprocessing
x = [process(xi) for xi in x_raw]
x_list = [xi.split() for xi in x]

# stopword elemination
stopw = readAFile('stopwords.txt')
stopw = np.array([s.replace('\n','') for s in stopw])
x_list = [[xij for xij in xi #if((xij not in stopw) #and len(xij)>2)\
                             ] for xi in x_list]
x_str = [' '.join(xi) for xi in x_list]

### exemple

In [32]:
print('* avant pre-processing:\n')
print(x_raw[11]+'\n\n')

print('* après pre-processing:\n')
print(x_str[11])

* avant pre-processing:

 Je ne sais ni pourquoi ni comment on s'est opposé il y a quelques douze années - douze ou treize ans - à la création de l'Université technologique.



* après pre-processing:

je ne sais ni pourquoi ni comment on s est oppose il y a quelques douze annees douze ou treize ans a la creation de l universite technologique


## du texte à la représentation vectorielle

In [19]:
countVe = CountVectorizer(max_df=0.78, min_df=1, ngram_range=(1,3))
count = countVe.fit_transform(x_str)

#Alternatives
#count = tfidf_vectorizer.fit_transform(x_str)
#tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, #max_features=20000,

X = count
y = np.array(y)

## application de modèles d'apprentissage

### naive bayes multinomial

In [59]:
nb_clf=sklearn.naive_bayes.MultinomialNB()
nb_clf.fit(X,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### svm avec noyau linéaire

In [52]:
C = .9  # SVM regularization parameter
svm_clf = svm.LinearSVC(C=C,max_iter=10000,tol=1e-6,class_weight={0:1.4,1:1.}).fit(X, y)

### modèle maxent

In [60]:
maxent_clf = sklearn.linear_model.LogisticRegression(max_iter=100,C=1.9)
maxent_clf.fit(X,y)

LogisticRegression(C=1.9, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### svm avec noyau linéaire

In [52]:
C = .9  # SVM regularization parameter
svm_clf = svm.LinearSVC(C=C,max_iter=10000,tol=1e-6,class_weight={0:1.4,1:1.}).fit(X, y)

## résultats des expériences


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Note**: Nous utilisons içi des données de validation pour confronter nos différents modèles et représentations vectorielles, afin d'éviter le phénomène "surapprentissage" qui pourrait apparaître lors de la modification des hyperparamètres des modèles pour se "coller" au données de test. L'utilisation de données de validation nous permet donc de vérifier si un modèle et/ou une représentation est efficace, et les données de test nous permettent d'y attester en mesurant le pouvoir de généralisation de ce modèle avec ces hyperparamètres. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Note bis**: Au lieu d'utiliser le taux de bonne prédiction pour évaluer nos modèles, nous utilisons le f1-score qui est une moyenne pondérée de la précision et du rappel:


>* **rappel**: $\Large\frac{\textrm{vrais positifs}}{\textrm{vrai positifs + faux négatifs}}$


>* **précision**: $\Large\frac{\textrm{vrais positifs}}{\textrm{vrais positifs + faux positifs}}$


### naive bayes multinomial

In [62]:
nb_scores = cross_validation.cross_val_score(nb_clf, X, y, scoring='f1_weighted')
print("Accuracy: %0.2f (+/- %0.2f)" % (nb_scores.mean(), nb_scores.std() * 2))

Accuracy: 0.86 (+/- 0.01)


### modèle maxent

In [61]:
maxent_scores = cross_validation.cross_val_score(maxent_clf, X, y, scoring='f1_weighted')
print("Accuracy: %0.2f (+/- %0.2f)" % (maxent_scores.mean(), maxent_scores.std() * 2))

Accuracy: 0.89 (+/- 0.00)


### svm avec noyau linéaire

In [57]:
svm_scores = cross_validation.cross_val_score(svm_clf, X, y, scoring='f1_weighted')
print("Accuracy: %0.2f (+/- %0.2f)" % (svm_scores.mean(), svm_scores.std() * 2))

Accuracy: 0.89 (+/- 0.00)


## chargement des données test et pre-processing

In [20]:
t = readAFile('test.utf8')

#extraction du texte
test = []
for txt in t:
    txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
    test.append(txt)    

#pre-processing
test = [process(xi) for xi in test]
test_list = [xi.split() for xi in test]
stopw = readAFile('stopwords.txt')
stopw = np.array([s.replace('\n','') for s in stopw])
test_list = [[xij for xij in xi #if(xij not in stopw #and len(xij)>2\
                                ] for xi in test_list]
test_str = [' '.join(xi) for xi in test_list]

### transformation en représentation sac de mots

In [39]:
count_test = countVe.transform(test_str)

## post-processing

In [40]:
def post(y, rang, coeff):
    '''for each label, we compute a count of the rang neighbors on each side, 
        and reassign the current label to the label that scored the highest
        on the weighted count -- using coeff.
        '''
    y_new=np.zeros((len(y)))
    for i in np.random.permutation(range(len(y))):
        findex = i - rang
        lindex = i + rang
        if(i - rang<0):
            findex = 0
            lindex = rang + i
        elif(i + rang > len(y)):
            findex = i - rang
            lindex = len(y) - 1
        countM = len(np.where(y[findex:lindex] == 0)[0])
        countC = len(np.where(y[findex:lindex] == 1)[0])
        y_new[i] = np.argmax([countM * coeff, countC])
    return y_new

## test du post-processing sur les données de validation

In [84]:
# dividing data in train and test, with ratio 0.4 for test data
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, \
                                        test_size=0.4, random_state=0)

### naive bayes multinomial

In [85]:
nb_clf.fit(X_train, y_train)
nb_preds = nb_clf.predict(X_test)
nb_preds_post = post(post(nb_preds,5,2.45),5,1.)
nb_score = f1_score(nb_preds, y_test, average='binary')
nb_score_post = f1_score(nb_preds_post, y_test, average='binary')
print("normal accuracy: "+str(nb_score))
print("accuracy after post-processing: "+str(nb_score_post))

normal accuracy: 0.936828021217
accuracy after post-processing: 0.928499590691


### modèle maxent

In [79]:
maxent_clf.fit(X_train, y_train)
maxent_preds = maxent_clf.predict(X_test)
maxent_preds_post = post(post(maxent_preds,5,2.45),5,1.)
maxent_score = f1_score(maxent_preds, y_test, average='binary')
maxent_score_post = f1_score(maxent_preds_post, y_test, average='binary')
print("normal accuracy: "+str(maxent_score))
print("accuracy after post-processing: "+str(maxent_score_post))

normal accuracy: 0.947266989473
accuracy after post-processing: 0.927039834843


### svm avec noyau linéaire

In [80]:
svm_clf.fit(X_train, y_train)
svm_preds = svm_clf.predict(X_test)
svm_preds_post = post(post(svm_preds,5,2.45),5,1.)
svm_score = f1_score(svm_preds, y_test, average='binary')
svm_score_post = f1_score(svm_preds_post, y_test, average='binary')
print("normal accuracy: "+str(svm_score))
print("accuracy after post-processing: "+str(svm_score_post))

normal accuracy: 0.944816865831
accuracy after post-processing: 0.922938644212


## prédictions des labels des données de test
### naive bayes multinomial

In [63]:
nb_pred_labels = nb_clf.predict(count_test)
nb_pred_labels = post(post(nb_pred_labels,5,2.45),5,1.)

### modèle maxent

In [64]:
maxent_pred_labels = maxent_clf.predict(count_test)
maxent_pred_labels = post(post(maxent_pred_labels,5,2.45),5,1.)

### svm avec noyau linéaire

In [66]:
svm_pred_labels = svm_clf.predict(count_test)
svm_pred_labels = post(post(svm_pred_labels,5,2.45),5,1.)

### exemple:

In [67]:
print svm_pred_labels[150:250]

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


## écriture du fichier des labels

In [47]:
f = open('test_labels_exemple.txt', 'w')
for i in pred_labels:
    if i == 1:
        f.write('C\n')
    else:
        f.write('M\n')
f.close()

## Commentaires


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Nous avons confrontés nos résultats sur les données de validations aux données de test, pour attester du pouvoir de généralisation des modèles et des représentations en donnant le document <code>test_labels_exemple.txt</code> au serveur, accessible depuis le lien suivant:
http://webia.lip6.fr/~guigue/wikihomepage/pmwiki.php?n=Course.CourseAFDSoumission



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Le modèle ayant le meilleur score sur les données de test est le svm (avec les paramètres définis plus haut). Nous avons utilisé une représentation sac de mots (sous forme de comptage des occurences), en en utilisant à la fois les uni-grams et les bi-grams. Il peut être utile de noter que lorsque l'on effectuait un pre-processing où l'on prenait trop de distance avec les données d'origine (lemmatisation des mots du document, ou prélevement des stopwords, par exemple), l'on avait quasi-systématiquement une baisse du score. Nous avons donc effectués un pre-preoccessing minimal.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Nous avons eu un score de **0.8011** au f1-score. Le post processing nous augmente notre score de manière non négligeable: nous avons des des augmentations de résultats de 15 à 20% au f1-score sur les données de test, même si nous n'avons pas observé une hausse du f1-score sur les données de validations (il est possible que les données d'apprentissage ont étés mélangés, et leur distribution (au sens de la répartition) ne colle plus à la distribution dans les données de test.)
